In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import os
import torchvision.models as tmodels
%matplotlib inline
torch.__version__

'1.7.1'

In [6]:
#data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])

BATCH_SIZE=32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_subset, val_subset = torch.utils.data.random_split(trainset, [40000, 10000])

trainloader = torch.utils.data.DataLoader(train_subset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(val_subset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [7]:
from model import Net
net = Net(BATCH_SIZE)

print(net)

net.fit(trainloader, 200, valloader)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
  (act): Softmax(dim=-1)
  (loss_fn): CrossEntropyLoss()
)


Done 100.0%, Epoch: 0, Loss: 1.4806303977966309, Accuracy: 43.75%: : 1250it [00:12, 101.30it/s]                


Avg loss: 1.4895156299591064, Avg accuracy: 46.099998474121094, Val loss: 0.04063269572854042, Val accuracy: 54.059999999999995
Improved val acc from -inf to 54.059999999999995, saving model


Done 100.0%, Epoch: 1, Loss: 1.1829822063446045, Accuracy: 59.375%: : 1250it [00:12, 99.73it/s]                


Avg loss: 1.230985656785965, Avg accuracy: 56.17499923706055, Val loss: 0.037600424873828885, Val accuracy: 57.269999999999996
Improved val acc from 54.059999999999995 to 57.269999999999996, saving model


...

Avg loss: 0.49921372148990634, Avg accuracy: 83.28250122070312, Val loss: 0.034265817427635195, Val accuracy: 65.09
Val acc did not improve from 65.99000000000001


Done 100.0%, Epoch: 198, Loss: 0.5875018239021301, Accuracy: 78.125%: : 1250it [00:09, 126.69it/s]                


Avg loss: 0.4986479867696762, Avg accuracy: 83.36750030517578, Val loss: 0.03386122795939445, Val accuracy: 65.42
Val acc did not improve from 65.99000000000001


Done 100.0%, Epoch: 199, Loss: 0.49412864446640015, Accuracy: 84.375%: : 1250it [00:09, 129.72it/s]               


Avg loss: 0.49854504874348643, Avg accuracy: 83.37749481201172, Val loss: 0.03415079381763935, Val accuracy: 65.42
Val acc did not improve from 65.99000000000001
Finished Training


In [8]:
acc, loss = net.test(testloader)
print(f'Test acc: {acc}%, test loss: {loss}')

Test acc: 66.05%, test loss: 0.03354053070545197
